In [4]:
# This Colab requires a recent numpy version.
!pip install python-dotenv

In [6]:
from dotenv import load_dotenv

In [7]:
load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")

# mydb = mysql.connector.connect(
#   host=DB_HOST,
#   user=DB_USER,
#   password=DB_PASSWORD,
#   database=DB_NAME,
#   port=3306
#)

False